# Applying logistic regression in python to discriminate between control and M. bovis infected animals using peripheral blood transcriptomics data from abdelaal et al., 2020
### This analysis considers two approaches, one using logistic regression on variable genes that have been preprocessed using DESeq2 (vst normalised) and the other using latent variables inferred using PCA, ICA and NMF

In [39]:
## Load in all necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from scipy import stats
from scipy.stats import kurtosis

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay,roc_curve,auc, make_scorer,mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score, PredefinedSplit
from sklearn.decomposition import PCA, NMF, FastICA
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
import warnings
import pickle
from sklearn.pipeline import Pipeline
import matplotlib
warnings.filterwarnings('ignore')

#font for plots
font = {'fontname':'Arial'}

In [40]:
abdelaal_data_raw = pd.read_csv("/home/workspace/jogrady/ML4TB/work/normalisation/vst_individual/abdelaal_vst_normalised_data.txt", sep = "\t").T

In [41]:
abdelaal_data_raw = pd.read_csv("/home/workspace/jogrady/ML4TB/work/normalisation/vst_individual/abdelaal_vst_normalised_data.txt", sep = "\t").T
abdelaal_labels = pd.read_csv("/home/workspace/jogrady/ML4TB/data/abdelaal/abdelaal_samples.csv", sep = "\t")
abdelaal_labels = abdelaal_labels[["Animal_Code", "Week", "Status"]].drop_duplicates()



np.array(abdelaal_labels["Animal_Code"]) == np.array(abdelaal_data_raw.index)
abdelaal_labels["ID"] = abdelaal_labels["Animal_Code"].str.extract(r"^(.*)_[^_]+$", expand=False)
abdelaal_labels["Fold"] = abdelaal_labels.groupby("ID").ngroup()
abdelaal_labels.loc[abdelaal_labels["Week"] == "W0", "Status"] = "Control" # Note the infected samples were sampled after infection (immediately) so may be worthwhile labelling them as Control (note this increased the CV accuracy from ~ 50 to 80)
# Check right order
np.array(abdelaal_labels["Animal_Code"]) == np.array(abdelaal_data_raw.index)

abdelaal_labels_simple = abdelaal_labels["Status"].to_numpy()
abdelaal_folds = abdelaal_labels["Fold"].to_numpy()


abdelaal_labels_simple = np.where(abdelaal_labels_simple == "Control", 0, np.where(abdelaal_labels_simple == "Infected", 1, abdelaal_labels_simple))
abdelaal_labels_simple = abdelaal_labels_simple.astype(int)

In [42]:
abdelaal_labels

,Animal_Code,Week,Status,ID,Fold
0,Infected_1_20,W20,Infected,Infected_1,0
2,Infected_1_8,W8,Infected,Infected_1,0
4,Infected_2_20,W20,Infected,Infected_2,1
6,Infected_2_8,W8,Infected,Infected_2,1
8,Infected_3_20,W20,Infected,Infected_3,2
10,Infected_3_8,W8,Infected,Infected_3,2
12,Infected_4_20,W20,Infected,Infected_4,3
14,Infected_4_8,W8,Infected,Infected_4,3
16,Infected_5_20,W20,Infected,Infected_5,4
18,Infected_5_8,W8,Infected,Infected_5,4


In [43]:
# Calcualte variances for VST normalised genes
variances = abdelaal_data_raw.var(axis=0)
# take top 20% and filter
threshold = variances.quantile(.80) 
genes = variances > threshold
genes= genes.loc[genes==True].index
abdelaal_data = abdelaal_data_raw.filter(items = genes, axis=1)

In [44]:
abdelaal_data.head()

,ENSBTAG00000054829,RCAN1,SMIM11,ITSN1,ABCG1,TFF2,TMPRSS3,RSPH1,IL1RAP,P3H2,...,ENSBTAG00000053997,ENSBTAG00000054086,ENSBTAG00000050585,ENSBTAG00000052194,SERPINB4,ENSBTAG00000050153,ENSBTAG00000050608,MAD2L1,ENSBTAG00000054081,ENSBTAG00000049569
Infected_1_20,5.020736,9.472897,8.265235,9.607765,10.199095,6.423200,5.446811,5.020736,8.269280,6.308950,...,4.543355,8.865097,4.933724,6.391624,4.543355,4.543355,6.238502,7.581310,6.291721,5.020736
Infected_1_8,4.824982,9.568001,8.278523,9.761322,10.372165,6.454131,5.738425,5.421722,8.782285,5.952382,...,4.543355,10.476969,4.941008,6.338420,4.941008,4.543355,6.500611,7.688112,7.109312,4.824982
Infected_2_20,4.543355,9.083210,8.664853,8.918923,10.963202,5.516665,5.000588,5.070595,8.262210,6.481582,...,4.543355,7.116487,6.936616,5.370276,5.000588,6.467869,5.482480,6.983877,6.337144,4.808073
Infected_2_8,5.016344,9.322084,8.500184,9.392623,10.808786,6.327843,5.209333,5.677687,9.055278,6.423233,...,4.543355,7.195843,6.634089,5.842797,5.355504,5.985045,5.707205,7.254159,6.110927,5.817100
Infected_3_20,4.924988,9.444165,9.058130,8.918001,11.077527,5.945029,5.501468,5.465131,8.492798,6.445798,...,4.543355,7.618384,7.150779,6.221806,5.081516,6.557695,4.924988,7.116096,5.720759,5.465131


In [45]:
# Convert the custom folds array to a PredefinedSplit object
ps = PredefinedSplit(test_fold=abdelaal_folds)

In [46]:

# Make a pipeline for logistic regression and set the paramaters
log_pipe = Pipeline(steps=[
('scaler', StandardScaler()), # see comment above (in markdown)
('classifier', LogisticRegression(max_iter=10000, solver='saga', tol=0.0001, random_state=42))]) # classifier

precision_scorer = make_scorer(precision_score, zero_division=1)  # had to modify zero_division as it was giving problems
f1_scorer = make_scorer(f1_score)
accuracy_scorer = make_scorer(accuracy_score)
recall_scorer = make_scorer(recall_score)

# Define scoring dictionary for GridSearchCV
scoring = {
    'accuracy': accuracy_scorer,
    #'f1': f1_scorer,
    #'precision': precision_scorer,
    #'recall': recall_scorer
}

# Create a parameter grid - we will search through all these combinations
param_grid = {
    'classifier__penalty': ["elasticnet"],
    'classifier__l1_ratio': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
}
 
# Create GridSearchCV object
grid_search = GridSearchCV(log_pipe, param_grid, cv=ps, verbose=3, n_jobs=60, scoring=scoring, refit="accuracy")

In [47]:
grid_search.fit(abdelaal_data, abdelaal_labels_simple)

Fitting 12 folds for each of 10 candidates, totalling 120 fits


[CV 2/12] END classifier__l1_ratio=0.1, classifier__penalty=elasticnet; accuracy: (test=0.500) total time=   9.4s
[CV 5/12] END classifier__l1_ratio=0.1, classifier__penalty=elasticnet; accuracy: (test=0.500) total time=   9.2s
[CV 10/12] END classifier__l1_ratio=0.2, classifier__penalty=elasticnet; accuracy: (test=1.000) total time=   8.7s
[CV 2/12] END classifier__l1_ratio=0.2, classifier__penalty=elasticnet; accuracy: (test=0.500) total time=   9.9s
[CV 11/12] END classifier__l1_ratio=0.3, classifier__penalty=elasticnet; accuracy: (test=0.000) total time=   9.6s
[CV 11/12] END classifier__l1_ratio=0.1, classifier__penalty=elasticnet; accuracy: (test=0.000) total time=  10.9s
[CV 3/12] END classifier__l1_ratio=0.1, classifier__penalty=elasticnet; accuracy: (test=0.000) total time=  11.1s
[CV 10/12] END classifier__l1_ratio=0.4, classifier__penalty=elasticnet; accuracy: (test=1.000) total time=  10.6s
[CV 7/12] END classifier__l1_ratio=0.1, classifier__penalty=elasticnet; accuracy: (t

GridSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,
        8,  9,  9, 10, 10, 11, 11])),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           random_state=42,
                                                           solver='saga'))]),
             n_jobs=60,
             param_grid={'classifier__l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6,
                                                  0.7, 0.8, 0.9, 1],
                         'classifier__penalty': ['elasticnet']},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score)}, verbose=3)

In [48]:
non_zero_counts = []
for params, mean_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_accuracy']):
    # Manually fit the pipeline with the parameters
    model = log_pipe.set_params(**params)
    model.fit(abdelaal_data, abdelaal_labels_simple)  # Re-fit the model on the entire dataset
    non_zero_count = np.sum(model.named_steps['classifier'].coef_ != 0)
    non_zero_counts.append((params, mean_score, non_zero_count))

non_zero_counts_df = pd.DataFrame(non_zero_counts)

non_zero_counts_df

non_zero_counts_df = non_zero_counts_df.rename(columns={0: "Paramaters", 1: "Average Accuracy", 2: "Non-zero coefficeints"})


# Look at best paramaters and accuracy and save results to a data frame
print("Best Parameters:", grid_search.best_params_)
print(f"Best CV average accuracy: {grid_search.best_score_:.3f}")
results_genes = pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),
           pd.DataFrame(grid_search.cv_results_["mean_test_accuracy"], columns = ["Average Accuracy"]),
           pd.DataFrame(grid_search.cv_results_["std_test_accuracy"], columns=["SD accuracy"])],axis=1)
results_genes.sort_values(by='Average Accuracy', inplace=True)
results_genes = pd.concat([results_genes, non_zero_counts_df.loc[:,["Non-zero coefficeints"]]], axis=1)
results_genes.sort_values(by='Average Accuracy', inplace=True)
results_genes

results_genes
results_models = pd.DataFrame(grid_search.cv_results_)
results_models.sort_values(by='rank_test_accuracy', inplace=True)
results_genes
results_genes.to_csv("/home/workspace/jogrady/ML4TB/results/Logistic_regression/CV_results/Abdelaal_gene_CV_results.txt")

Best Parameters: {'classifier__l1_ratio': 0.1, 'classifier__penalty': 'elasticnet'}
Best CV average accuracy: 0.667


In [49]:
results_genes

,classifier__l1_ratio,classifier__penalty,Average Accuracy,SD accuracy,Non-zero coefficeints
6,0.7,elasticnet,0.500000,0.408248,85
7,0.8,elasticnet,0.500000,0.408248,70
8,0.9,elasticnet,0.500000,0.408248,61
4,0.5,elasticnet,0.541667,0.431003,123
5,0.6,elasticnet,0.541667,0.431003,97
9,1.0,elasticnet,0.541667,0.379601,52
3,0.4,elasticnet,0.583333,0.399653,167
2,0.3,elasticnet,0.625000,0.360844,234
0,0.1,elasticnet,0.666667,0.372678,695
1,0.2,elasticnet,0.666667,0.372678,358


In [50]:
params = {'classifier__l1_ratio': 0.2, 'classifier__penalty': 'elasticnet'}
model = log_pipe.set_params(**params)
model.fit(abdelaal_data, abdelaal_labels_simple)  # Re-fit the model on the entire dataset
# Save model in case we ever need it again e.g. for external data
with open('/home/workspace/jogrady/ML4TB/work/models/Logistic_regression/Abdelaal_Logistic_regression_l10.2_search.pkl', 'wb') as f:
    pickle.dump(model, f)

# PCA

In [51]:
 
# Do not need to scale for PCA - not recommended in VST
pca = PCA(random_state=88, n_components=24)
pca.fit(abdelaal_data)
explained_variance = pca.explained_variance_ratio_
# Plotting the elbow curve
plt.figure(figsize=(15, 6))
plt.plot(range(1, 24+1), explained_variance, marker='o')
plt.xlabel('Number of Principal Components based on training set')
plt.ylabel('Variance explained')
plt.title('Elbow Method for Optimal Number of Components in training set')
plt.grid()
plt.xticks(range(1, 24 + 1))
plt.axvline(x=6, color='r', linestyle='--', label='Optimal Components (based on elbow)')
plt.axvline(x=9, color='g', linestyle='--', label='80% variance captured')
plt.legend()
plt.show()
print(f'Variance captured by 9 PCs:{pca.explained_variance_ratio_[:9].sum():.2f}')

Variance captured by 9 PCs:0.82


In [52]:
# Set up the pipeline
PCA_Pipeline = Pipeline(steps=[('pca', PCA(random_state=42)),
('classifier', LogisticRegression(max_iter=10000, penalty="none", solver='saga', tol=0.0001, random_state=42))])

# Set up a grid for each PC
pca_param_grid = {'pca__n_components': list(range(1, 10))}

# Apply
LR_pca_search_model = GridSearchCV(PCA_Pipeline, pca_param_grid, cv=ps, scoring=scoring, refit="accuracy")

# Fit
LR_pca_search_model.fit(abdelaal_data, abdelaal_labels_simple.ravel())

GridSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,
        8,  9,  9, 10, 10, 11, 11])),
             estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='none',
                                                           random_state=42,
                                                           solver='saga'))]),
             param_grid={'pca__n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score)})

In [53]:
# Look at best paramaters and accuracy and save results to a data frame
print(f"Best Paramater:", LR_pca_search_model.best_params_)
print(f"Best Score: {LR_pca_search_model.best_score_:.2f}")
results_pca = pd.concat([pd.DataFrame(LR_pca_search_model.cv_results_["params"]),
           pd.DataFrame(LR_pca_search_model.cv_results_["mean_test_accuracy"], columns = ["CV Accuracy"]),
           pd.DataFrame(LR_pca_search_model.cv_results_["std_test_accuracy"], columns=["SD accuracy"])],axis=1)
results_pca.sort_values(by='CV Accuracy', inplace=True)

Best Paramater: {'pca__n_components': 9}
Best Score: 0.75


In [54]:
with open('/home/workspace/jogrady/ML4TB/work/models/Logistic_regression/Abdelaal_Logistic_regression_PCA_search.pkl', 'wb') as f:
    pickle.dump(LR_pca_search_model, f)

In [55]:
print(results_pca)
results_pca.to_csv(path_or_buf="/home/workspace/jogrady/ML4TB/results/Logistic_regression/CV_results/Abdelaal_PCA_CV_results.txt", sep = "\t")

   pca__n_components  CV Accuracy  SD accuracy
0                  1     0.041667     0.138193
3                  4     0.416667     0.343592
1                  2     0.458333     0.379601
2                  3     0.500000     0.353553
6                  7     0.500000     0.408248
5                  6     0.541667     0.431003
4                  5     0.583333     0.343592
7                  8     0.625000     0.414578
8                  9     0.750000     0.381881


In [56]:
# 'residuals_all.columns' contains the names of the genes

pca = PCA(random_state=42, n_components=10)
pca_fit = pca.fit(abdelaal_data)
comp_genes= []
pca_genes = pd.DataFrame()
pca_results = pca_fit.fit_transform(abdelaal_data)
colors = ['steelblue' if label == 0 else 'crimson' for label in abdelaal_labels_simple.ravel()]
plt.scatter(pca_results[:, 0], pca_results[:, 1], label="Training data", c=colors, s=20)
handles = [plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='steelblue', markersize=5, label='Control'),
           plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='crimson', markersize=5, label='Infected')]
plt.title("PCA Visualization of eigenvectors", fontweight='bold')
plt.legend(handles=handles)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()
# Iterate through each component
for component_idx in range(10):
    # Calculate loadings for the current component
    loadings = pca_fit.components_.T * np.sqrt(pca_fit.explained_variance_)

    # Sort genes by their influence on the current component
    sorted_genes = abdelaal_data.columns[np.argsort(loadings[:, component_idx])]

    # Visualize the top N genes that contribute the most to the current component
    N = pca_fit.components_.shape[1]
    top_genes = sorted_genes[-N:]

    pca_genes[component_idx] = top_genes[::-1]
    comp_genes = np.append(comp_genes, top_genes)
pca_genes.columns = ["Component_1","Component_2","Component_3","Component_4",
                     "Component_5","Component_6","Component_7","Component_8",
                     "Component_9","Component_10"]                      
pca_genes 

,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,Component_9,Component_10
0,ENSBTAG00000039256,STEAP4,ENSBTAG00000054494,ENSBTAG00000031825,PRSS2,PRSS2,ENSBTAG00000052325,ENSBTAG00000008328,ADAM11,ENSBTAG00000054947
1,ENSBTAG00000038080,SLC1A1,ENSBTAG00000006383,ENSBTAG00000025398,RBM44,ENSBTAG00000040367,ENSBTAG00000011961,ENSBTAG00000048135,ENSBTAG00000050515,CDH2
2,ENSBTAG00000013055,ENSBTAG00000049271,ENSBTAG00000031825,ENSBTAG00000038080,ENSBTAG00000051519,FXYD4,ENSBTAG00000053635,PRSS2,WNT5A,ENSBTAG00000052233
3,ALOX15,ENSBTAG00000055309,ENSBTAG00000051896,ENSBTAG00000013055,ENSBTAG00000039256,TUBB1,TUBB1,TMC7,IQSEC3,MRPS25
4,ENSBTAG00000006383,ENSBTAG00000053356,KLRF2,TACR3,SYT7,PDLIM1,MPIG6B,ENSBTAG00000019017,ENSBTAG00000054705,OSBP2
...,...,...,...,...,...,...,...,...,...,...
5517,ENSBTAG00000052428,RBM44,RADX,ENSBTAG00000054947,bta-mir-11979,BATF2,CACNA1B,ENSBTAG00000050510,ENSBTAG00000052514,PGA5
5518,SLC8A1,ENSBTAG00000050585,CBLIF,ENSBTAG00000037421,ENSBTAG00000052233,ENSBTAG00000045580,ENSBTAG00000038080,IFITM1,ENSBTAG00000006383,ENSBTAG00000037539
5519,TGFA,CERS6,ENSBTAG00000037421,ENSBTAG00000040367,BOLA-DQA5,TNFAIP2,BOLA-DQA5,ENSBTAG00000045580,LRP11,ENSBTAG00000055185
5520,STEAP4,MRPS25,BOLA-DQA5,BOLA-DQA5,ENSBTAG00000009656,ENSBTAG00000054705,CBLIF,RADX,ENSBTAG00000055211,RADX


In [57]:
# Set up the grid for subplots (10 rows, 3 columns)
fig, axes = plt.subplots(3, 3, figsize=(18, 30))
fig.suptitle("PCA Visualization of Eigenvectors", fontweight='bold', fontsize=16)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop to generate scatter plots from PC1 vs each PC up to PC30
for i in range(1, 10):
    ax = axes[i-1]
    ax.scatter(pca_results[:, 0], pca_results[:, i], c=colors, s=20)
    ax.set_title(f"PC1 vs PC{i+1}")
    ax.set_xlabel("PC1")
    ax.set_ylabel(f"PC{i+1}")

# Custom legend only in the first subplot (for cleanliness)
handles = [
    plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='steelblue', markersize=5, label='Control'),
    plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='crimson', markersize=5, label='Infected')
]
axes[0].legend(handles=handles)

# Hide any unused subplots (since there are only 29 plots for a 30-slot grid)
for j in range(29, len(axes)):
    axes[j].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit title
plt.show()

In [58]:
Feature_weight_pca = pd.DataFrame(zip(pca_genes.columns, np.transpose(LR_pca_search_model.best_estimator_.named_steps["classifier"].coef_)), columns=['features', 'coef'])#.sort_values(by='coef', inplace=True)
Feature_weight_pca.sort_values(by='coef', inplace = True)
Feature_weight_pca = Feature_weight_pca.loc[(Feature_weight_pca != 0).all(axis=1), :]
Feature_weight_pca
# PC 5 is very important in the model

,features,coef
8,Component_9,[-1.536679067820141]
2,Component_3,[-0.9522225560500457]
6,Component_7,[-0.5054857892466998]
5,Component_6,[-0.3839239904971617]
0,Component_1,[-0.10098649834738362]
3,Component_4,[0.1532031343030736]
1,Component_2,[0.2458453043305297]
7,Component_8,[0.29649556847883085]
4,Component_5,[0.44264740413875125]


In [59]:
df_pca = pd.DataFrame(pca_results)
Control = df_pca.iloc[:8, :] # 45 samples in control
Infected = df_pca.iloc[9:, :] # 42 in infected

# Perform t-test for each component
results = pd.DataFrame(columns=['Component', 'T-Statistic', 'P-Value'])

for column in df_pca.columns:
    t_statistic, p_value = stats.ttest_ind(Control[column], Infected[column])
    
    new_row = pd.DataFrame({'Component': [column], 'T-Statistic': [t_statistic], 'P-Value': [p_value]})
    results = pd.concat([results, new_row], ignore_index=True)

# Print the results
print(results)

# Identify components with significantly different means

from statsmodels.stats.multitest import multipletests


reject, pvals_corrected, _, _ = multipletests(results["P-Value"], alpha=0.05, method='fdr_bh')
results['Corrected P-Value'] = pvals_corrected
results['Significant (BH)'] = reject 


significantly_different = results[results['Corrected P-Value'] < 0.05]
print("Components with significantly different means:")
print(significantly_different)

  Component  T-Statistic   P-Value
0         0    -0.033151  0.973867
1         1     1.572100  0.130873
2         2    -1.988008  0.060004
3         3     0.243318  0.810120
4         4    -0.297891  0.768713
5         5    -2.306678  0.031360
6         6     0.744525  0.464809
7         7     0.191887  0.849674
8         8    -0.175597  0.862293
9         9    -0.562930  0.579444
Components with significantly different means:
Empty DataFrame
Columns: [Component, T-Statistic, P-Value, Corrected P-Value, Significant (BH)]
Index: []


### 2. ICA

ICA and PCA are similar to each other however ICA attemps to transform the data into statistically significant non-Gaussian components. Often times this is estimated by Kurtosis but the number of  !'PCs'! used for the whitening procedure is often those that reach 80,90,95% etc, hence we will use this for below  

In [60]:
# ICA
from sklearn.decomposition import FastICA
ICA_transformer = FastICA(n_components=9, # from PCA - 80% of variance
        random_state=42,
        max_iter=1000, tol=0.0001,
        whiten='unit-variance')

df_train_ica = ICA_transformer.fit_transform(abdelaal_data)
kurtosis_scores = [kurtosis(df_train_ica[:, i]) for i in range(df_train_ica.shape[1])]
n_components = np.argmax(kurtosis_scores) + 1

In [61]:
# Number of non-normal components in training data = 3 - however we will use the same as above for PCA
n_components

5

In [62]:
# Set up the pipeline
ICA_Pipeline = Pipeline(steps=[('ica', FastICA( 
        random_state=42,
        max_iter=5000, tol=0.0001,
        whiten='unit-variance')),
('classifier', LogisticRegression(max_iter=10000, penalty="none", solver='saga', tol=0.0001, random_state=42))])

# Set up grid of components - 80% from PCA above
ica_param_grid = {'ica__n_components': list(range(1, 11))}


# Apply
LR_ica_search_model = GridSearchCV(ICA_Pipeline, ica_param_grid, cv=ps, scoring=scoring, refit="accuracy")


# Fit
LR_ica_search_model.fit(abdelaal_data, abdelaal_labels_simple.ravel())


GridSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,
        8,  9,  9, 10, 10, 11, 11])),
             estimator=Pipeline(steps=[('ica',
                                        FastICA(max_iter=5000,
                                                random_state=42)),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='none',
                                                           random_state=42,
                                                           solver='saga'))]),
             param_grid={'ica__n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score)})

In [63]:
# Look at best paramaters and accuracy and save results to a data frame
print("Best Parameters:", LR_ica_search_model.best_params_)
print(f"Best Score: {LR_ica_search_model.best_score_:.2f}")
results_ica = pd.concat([pd.DataFrame(LR_ica_search_model.cv_results_["params"]),
           pd.DataFrame(LR_ica_search_model.cv_results_["mean_test_accuracy"], columns = ["CV Accuracy"]),
           pd.DataFrame(LR_ica_search_model.cv_results_["std_test_accuracy"], columns=["SD accuracy"])],axis=1)
results_ica.sort_values(by='CV Accuracy', inplace=True)

with open('/home/workspace/jogrady/ML4TB/work/models/Logistic_regression/Abdelaal_Logistic_regression_ICA_search.pkl', 'wb') as f:
    pickle.dump(LR_ica_search_model, f)
    
print(results_ica)
results_ica.to_csv(path_or_buf="/home/workspace/jogrady/ML4TB/results/Logistic_regression/CV_results/Abdelaal_ICA_CV_results.txt", sep = "\t")

Best Parameters: {'ica__n_components': 9}
Best Score: 0.71
   ica__n_components  CV Accuracy  SD accuracy
0                  1     0.000000     0.000000
1                  2     0.208333     0.246503
6                  7     0.458333     0.431003
5                  6     0.500000     0.456435
2                  3     0.541667     0.379601
3                  4     0.541667     0.379601
4                  5     0.583333     0.343592
9                 10     0.583333     0.399653
7                  8     0.625000     0.414578
8                  9     0.708333     0.379601


### 3. Non-negative matrix factorization

In [64]:
NMF_Pipeline = Pipeline(steps=[('nmf', NMF(
    init="random", solver='cd', 
    beta_loss='frobenius', tol=0.0001, 
    max_iter=10000, random_state=42, verbose=0)),
('classifier', LogisticRegression(max_iter=10000, penalty=None, solver='saga', tol=0.0001, random_state=42))])

nmf_param_grid = {'nmf__n_components': list(range(1, 11))}

LR_nmf_search_model = GridSearchCV(NMF_Pipeline, nmf_param_grid, cv=ps, n_jobs= 10, scoring=scoring, refit="accuracy")

LR_nmf_search_model.fit(abdelaal_data, abdelaal_labels_simple.ravel())

GridSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,
        8,  9,  9, 10, 10, 11, 11])),
             estimator=Pipeline(steps=[('nmf',
                                        NMF(init='random', max_iter=10000,
                                            random_state=42)),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty=None,
                                                           random_state=42,
                                                           solver='saga'))]),
             n_jobs=10,
             param_grid={'nmf__n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score)})

In [65]:
# Look at best paramaters and accuracy and save results to a data frame
print("Best Parameters:", LR_nmf_search_model.best_params_)
print(f"Best Score: {LR_nmf_search_model.best_score_:.2f}")
results_nmf = pd.concat([pd.DataFrame(LR_nmf_search_model.cv_results_["params"]),
           pd.DataFrame(LR_nmf_search_model.cv_results_["mean_test_accuracy"], columns = ["CV Accuracy"]),
           pd.DataFrame(LR_nmf_search_model.cv_results_["std_test_accuracy"], columns=["SD accuracy"])],axis=1)
results_nmf.sort_values(by='CV Accuracy', inplace=True)

print(results_nmf)
results_nmf.to_csv(path_or_buf="/home/workspace/jogrady/ML4TB/results/Logistic_regression/CV_results/Abdelaal_NMF_CV_results.txt", sep = "\t")
with open('/home/workspace/jogrady/ML4TB/work/models/Logistic_regression/Abdelaal_Logistic_regression_NMF_search.pkl', 'wb') as f:
    pickle.dump(LR_nmf_search_model, f)

Best Parameters: {'nmf__n_components': 9}
Best Score: 0.62
   nmf__n_components  CV Accuracy  SD accuracy
0                  1     0.000000     0.000000
1                  2     0.000000     0.000000
2                  3     0.291667     0.320048
7                  8     0.416667     0.448764
4                  5     0.500000     0.353553
6                  7     0.500000     0.456435
9                 10     0.500000     0.353553
3                  4     0.541667     0.379601
5                  6     0.583333     0.343592
8                  9     0.625000     0.414578


## Evaluation
# Ogrady_data_test

In [66]:
ensemble = pd.read_csv(
    "/home/workspace/jogrady/eqtl_study/eqtl_nextflow/data/RNA_seq/Bos_taurus.ARS-UCD1.2.110.gtf",
    sep="\t",
    header=None,
    comment="#",
)
# Filter for rows where column 3 (V3) is "gene"
ensemble = ensemble[ensemble[2] == "gene"]

# Split column 9 (V9) into separate columns for gene_id, gene_version, and gene_name
ensemble_split = ensemble[8].str.split(";", expand=True)
ensemble["gene_id"] = ensemble_split[0].str.replace("^gene_id ", "", regex=True).str.replace('"', '', regex=False)
ensemble["gene_version"] = ensemble_split[1]
ensemble["gene_name"] = ensemble_split[2].str.replace("gene_name ", "", regex=True).str.replace('"', '', regex=False)

# Clean up the gene_name column
ensemble["gene_name"] = (
    ensemble["gene_name"]
    .str.replace("gene_source ", "", regex=False)
    .str.strip()
)
ensemble["gene_name"] = np.where(
    ensemble["gene_name"].isin(["ensembl", "5S_rRNA"]), 
    ensemble["gene_id"], 
    ensemble["gene_name"]
)

# Rename columns
ensemble.rename(columns={0: "chr"}, inplace=True)
ensemble = ensemble[["gene_id", "gene_name", "chr", 3]]
ensemble.rename(columns={3: "pos"}, inplace=True)

# Select only gene_id and gene_name
ensemble = ensemble[["gene_id", "gene_name"]]

# Handle duplicated gene_name values
ensemble["gene_name"] = np.where(
    ensemble["gene_name"].duplicated(), 
    ensemble["gene_id"], 
    ensemble["gene_name"]
)

# Clean up gene_id and gene_name columns for alignment
ensemble["gene_name"] = ensemble["gene_name"].str.replace(' ', '', regex=False)
ensemble["gene_id"] = ensemble["gene_id"].str.replace(' ', '', regex=False)


In [67]:
ogrady_test = pd.read_csv("/home/workspace/jogrady/ML4TB/work/normalisation/vst_combined/ogradytest_abdelaal_vst_normalised_data.txt", sep = "\t")

tested_genes = pd.DataFrame(ogrady_test.index)
tested_genes.columns = ["gene_id"]

tested_genes = tested_genes.merge(ensemble, how="left", on="gene_id")
ogrady_test.index = tested_genes["gene_name"].to_numpy()
ogrady_test = ogrady_test.T


test_labels = pd.read_csv("/home/workspace/jogrady/ML4TB/work/normalisation/vst_individual/Test_labels.txt", sep = "\t").T.to_numpy() 
test_labels = np.where(test_labels == "Control", 0, np.where(test_labels == "Infected", 1, test_labels))
test_labels = test_labels.astype(int)
# Calcualte variances for VST normalised genes
variances = abdelaal_data_raw.var(axis=0)
# take top 20% and filter
threshold = variances.quantile(.80) 
genes = variances > threshold
genes= genes.loc[genes==True].index
#print(genes)
ogrady_test = ogrady_test.filter(items = genes, axis = 1)

In [68]:
target_names = ['Control', 'Infected']
report = classification_report(test_labels.ravel(), model.predict(ogrady_test), target_names=target_names, output_dict=True)
df_report = pd.DataFrame(report).transpose()
Gene_cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(test_labels.ravel(),model.predict(ogrady_test)), display_labels = ["Control", "Infected"])

In [69]:
print(classification_report(test_labels.ravel(), model.predict(ogrady_test), target_names=target_names))

              precision    recall  f1-score   support

     Control       0.00      0.00      0.00        18
    Infected       0.50      1.00      0.67        18

    accuracy                           0.50        36
   macro avg       0.25      0.50      0.33        36
weighted avg       0.25      0.50      0.33        36



In [70]:
# Performance on Test data
print(classification_report(test_labels.ravel(), LR_pca_search_model.predict(ogrady_test), target_names=target_names))
PCA_cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(test_labels.ravel(),LR_pca_search_model.predict(ogrady_test)), display_labels = ["Control", "Infected"])
#ogrady_master_df_predictions = pd.concat([ogrady_master_df_predictions, pd.DataFrame({"Logistic_PCA_prediction": LR_pca_search_model.predict(ogrady_data)})], axis = 1)

              precision    recall  f1-score   support

     Control       0.48      0.89      0.63        18
    Infected       0.33      0.06      0.10        18

    accuracy                           0.47        36
   macro avg       0.41      0.47      0.36        36
weighted avg       0.41      0.47      0.36        36



In [33]:
# Performance on Test data
print(classification_report(test_labels.ravel(), LR_ica_search_model.predict(ogrady_test), target_names=target_names))
PCA_cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(test_labels.ravel(),LR_ica_search_model.predict(ogrady_test)), display_labels = ["Control", "Infected"])
#ogrady_master_df_predictions = pd.concat([ogrady_master_df_predictions, pd.DataFrame({"Logistic_PCA_prediction": LR_pca_search_model.predict(test_data)})], axis = 1)

              precision    recall  f1-score   support

     Control       0.48      0.89      0.63        18
    Infected       0.33      0.06      0.10        18

    accuracy                           0.47        36
   macro avg       0.41      0.47      0.36        36
weighted avg       0.41      0.47      0.36        36



In [34]:
# Performance on Test data
print(classification_report(test_labels.ravel(), LR_nmf_search_model.predict(ogrady_test), target_names=target_names))
PCA_cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(test_labels.ravel(),LR_nmf_search_model.predict(ogrady_test)), display_labels = ["Control", "Infected"])
#ogrady_master_df_predictions = pd.concat([ogrady_master_df_predictions, pd.DataFrame({"Logistic_PCA_prediction": LR_pca_search_model.predict(test_data)})], axis = 1)

              precision    recall  f1-score   support

     Control       0.49      0.94      0.64        18
    Infected       0.00      0.00      0.00        18

    accuracy                           0.47        36
   macro avg       0.24      0.47      0.32        36
weighted avg       0.24      0.47      0.32        36



In [71]:
# predict continuous value
# For different decision thresholds
y_score_gene = model.predict_proba(ogrady_test)
fprG_gene, tprG_gene, t_gene = roc_curve(test_labels.ravel(), y_score_gene[:,1])
roc_aucG_gene = auc(fprG_gene, tprG_gene)


y_score_pca = LR_pca_search_model.predict_proba(ogrady_test)
fprG_pca, tprG_pca, t_pca = roc_curve(test_labels.ravel(), y_score_pca[:,1])
roc_aucG_pca = auc(fprG_pca, tprG_pca)


y_score_ica = LR_ica_search_model.predict_proba(ogrady_test)
fprG_ica, tprG_ica, t_ica = roc_curve(test_labels.ravel(), y_score_ica[:,1])
roc_aucG_ica = auc(fprG_ica, tprG_ica)



y_score_nmf = LR_nmf_search_model.predict_proba(ogrady_test)
fprG_nmf, tprG_nmf, t_nmf = roc_curve(test_labels.ravel(), y_score_nmf[:,1])
roc_aucG_nmf = auc(fprG_nmf, tprG_nmf)

In [72]:
matplotlib.use('cairo')

In [73]:
#%matplotlib inline
plt.figure()
lw = 2
plt.plot(fprG_gene, tprG_gene, color='red',
         lw=lw, label='ROC Gene (area = %0.3f)' % roc_aucG_gene)
plt.plot(fprG_pca, tprG_pca, color='yellow',
         lw=lw, label='ROC PCA (area = %0.3f)' % roc_aucG_pca)
plt.plot(fprG_ica, tprG_ica, color='orange',
         lw=lw, label='ROC ICA (area = %0.3f)' % roc_aucG_ica)
plt.plot(fprG_nmf, tprG_nmf, color='blue',
         lw=lw, label='ROC NMF (area = %0.3f)' % roc_aucG_nmf)

plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("ROC analysis with different models from abdelaal et al\non O'Grady et al. hold out set")
plt.legend(loc="lower right")
plt.savefig("/home/workspace/jogrady/ML4TB/results/Logistic_regression/CV_results/Abdelaal_train_V_OGrady_test_ROC.pdf", format="pdf", bbox_inches="tight")
plt.show()
